## Data Preparation

In [242]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [244]:
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:34807 Dashboard: http://127.0.0.1:37885/status,Cluster Workers: 4 Cores: 4 Memory: 16.63 GB


### Database Connection

In [205]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [206]:
properties.count_documents({})

14209

### Convert Mongo Collection to DataFrame

#### New Properties

In [207]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [208]:
df_general_info = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [209]:
df_location = json_normalize(new_properties_json, record_path='location', meta='urlProperty')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [210]:
df_owner_property = json_normalize(new_properties_json, record_path='ownerProperty', meta='urlProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [211]:
df_features = json_normalize(new_properties_json, record_path='features', meta='urlProperty')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [212]:
ddf_features = ddf_features.rename(columns={
    'price':'range_prices',
    'rooms': 'general_rooms',
    'bathrooms': 'general_bathrooms',
    'privateArea': 'range_private_area'
})

In [213]:
df_more_features = json_normalize(new_properties_json, record_path='moreFeatures', meta='urlProperty')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [214]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
for ddf in ddf_list:
    ddf.set_index('urlProperty')

ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,property,offerType,area,privateArea,rooms,bathrooms,price,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,country,department,city,sector,neighborhood,address,latitude,longitude,urlProperty,id,name,contractType,financing,schedule,urlProperty,range_prices,squareMeters,general_rooms,general_bathrooms,garages,range_private_area,constructionArea,squareMetersPrice,stratum,condition,antiquity,floor,interiorFloors,weather,includesAdministration,admonPrice,urlProperty,interiorFeatures,exteriorFeatures,sectorFeatures,urlProperty
npartitions=21,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,object,int64,object,object,object,object,object,object,object,int64,int64,object,object,object,float64,object,object,object,object,int64,object,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [215]:
ddf_new_properties = ddf.loc[:,~ddf.columns.duplicated()]

#### Old Properties

In [216]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [217]:
df_general_info = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [218]:
df_location = json_normalize(old_properties_json, 'location')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [219]:
df_owner_property = json_normalize(old_properties_json, 'ownerProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [220]:
df_features = json_normalize(old_properties_json, 'features')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [221]:
df_more_features = json_normalize(old_properties_json, 'moreFeatures')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [222]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,country,department,city,sector,neighborhood,address,latitude,longitude,id,name,contractType,financing,schedule,price,squareMeters,rooms,bathrooms,garages,privateArea,constructionArea,squareMetersPrice,stratum,condition,antiquity,floor,interiorFloors,weather,includesAdministration,admonPrice,interiorFeatures,exteriorFeatures,sectorFeatures
npartitions=20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,float64,float64,int64,object,object,object,object,object,object,int64,int64,object,object,object,float64,object,object,object,object,int64,object,object,float64,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [223]:
ddf_old_properties = ddf.loc[:,~ddf.columns.duplicated()]

### Rename DataFrame Columns

#### New Properties

In [224]:
ddf_new_properties.columns

Index(['property', 'offerType', 'area', 'privateArea', 'rooms', 'bathrooms',
       'price', '_id', 'urlProperty', 'scrapingDate', 'scrapingHour',
       'modifyDate', 'modifyHour', 'code', 'status', 'type', 'use',
       'nameProject', 'description', 'country', 'department', 'city', 'sector',
       'neighborhood', 'address', 'latitude', 'longitude', 'id', 'name',
       'contractType', 'financing', 'schedule', 'range_prices', 'squareMeters',
       'general_rooms', 'general_bathrooms', 'garages', 'range_private_area',
       'constructionArea', 'squareMetersPrice', 'stratum', 'condition',
       'antiquity', 'floor', 'interiorFloors', 'weather',
       'includesAdministration', 'admonPrice', 'interiorFeatures',
       'exteriorFeatures', 'sectorFeatures'],
      dtype='object')

In [234]:
ddf_new_properties = ddf_new_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "range_square_meters",
                        "constructionArea": "range_construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features",
                        "offerType": "offer_type",
                        "privateArea": "private_area"
})

#### Old Properties

In [235]:
ddf_old_properties = ddf_old_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "square_meters",
                        "privateArea": "private_area",
                        "constructionArea": "construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features"
})

### Data Cleaning

#### New Properties

In [236]:
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [237]:
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters)), meta=('square_meters', 'float'))
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area)), meta=('private_area', 'float'))
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area)), meta=('construction_area', 'float'))
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

### Testing

#### New Properties

In [245]:
ddf_new_properties.head(10)

,property,offer_type,area,private_area,rooms,bathrooms,price,id_mongoose,id_property,scraping_date,...,condition,antiquity,floor,interior_floors,weather,includes_administration,admon_price,interior_features,exterior_features,sector_features
0,Apartamento,Venta,46.3,0.0,2,2,206200000,5dbaf08d8f40fc05710379ee,https://www.fincaraiz.com.co/reserva-serrat-se...,31/10/2019,...,,,0,0,,False,0.0,"[Ascensores Comunales, Balcón, Closet]","[Piscina, Salón Comunal, Zona de Camping, Zona...","[Cómodas vias de acceso, Seguridad, Supermerca..."
1,Apartamento,Venta,57.9,0.0,3,2,238400000,5dbaf08d8f40fc05710379ee,https://www.fincaraiz.com.co/reserva-serrat-se...,31/10/2019,...,,,0,0,,False,0.0,"[Balcón, Estudio, Garaje Cubierto]","[Ascensor, Cancha de Squash, Garaje / Parquead...",[Zona Residencial]
2,Apartamento,Venta,87.5,82.9,3,3,395632500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,,False,0.0,"[Balcón, Cocina Integral, Sauna / Turco / Jacu...","[Ascensor, En conjunto cerrado, Garaje / Parqu...","[Bombas de gasolina, Cerca centro comercial, C..."
3,Apartamento,Venta,87.5,82.9,3,3,406132500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,Cálido,False,0.0,"[Acceso para camiones, Acceso para tractomulas...","[Acceso Pavimentado, Corrales, Cuarto de Escol...","[Bombas de gasolina, Cerca a sector comercial,..."
4,Apartamento,Venta,87.5,82.9,3,3,396632500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,,False,0.0,[Balcón],"[Piscina, Salón Comunal]","[Cómodas vias de acceso, Restaurantes, Segurid..."
5,Apartamento,Venta,87.5,82.9,3,3,411632500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,,False,0.0,"[Balcón, Comedor]","[Ascensor, Salón de Juegos]","[Cerca centro comercial, Cómodas vias de acces..."
6,Apartamento,Venta,87.5,82.9,3,3,406632500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,Templado,False,0.0,"[Balcón, Citófono, Closet]","[Ascensor, Garaje / Parqueadero(s)]","[Bombas de gasolina, Cerca centro comercial, C..."
7,Apartamento,Venta,87.5,82.9,3,3,406632500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,Templado,False,0.0,"[Balcón, Comedor, Sauna / Turco / Jacuzzi, Zon...","[Gimnasio, Zona Infantil]","[Colegios / Universidades, Supermercados / C.C..."
8,Apartamento,Venta,87.5,82.9,3,3,407132500,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,,False,0.0,"[Balcón, Baño Auxiliar, Closet, Comedor]","[Ascensor, Garaje / Parqueadero(s), Piscina, S...","[Cómodas vias de acceso, Seguridad, Trans. Púb..."
9,Apartamento,Venta,87.5,82.9,3,3,397000000,5dbaf08db187b10d230379ee,https://www.fincaraiz.com.co/citte/medellin/pr...,31/10/2019,...,,,0,0,Cálido,False,0.0,"[Acceso para camiones, Acceso para tractomulas...","[Acceso Pavimentado, Corrales, Cuarto de Escol...","[Bombas de gasolina, Cerca a sector comercial,..."


#### Old Properties

In [246]:
ddf_old_properties.head(10)

,id_mongoose,id_property,scraping_date,scraping_hour,modify_date,modify_hour,code,active,type,new_property,...,condition,antiquity,floor,interior_floors,weather,includes_administration,admon_price,interior_features,exterior_features,sector_features
0,5dbaf08da724eb62520379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,19/10/2019,16:22:31,4868887,True,Venta,False,...,Excelente,,5,0,,False,0.0,"[Baño Auxiliar, Citófono, Instalación de gas, ...","[Circuito cerrado de TV, Portería / Recepción,...","[Colegios / Universidades, Parques cercanos, S..."
1,5dbaf08dae9d7e77530379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,30/09/2019,01:20:27,4913705,True,Venta,False,...,,Menos de 1 año,0,0,,False,335000.0,"[Balcón, Calentador, Closet, Cuarto de Servici...","[En conjunto cerrado, Garaje / Parqueadero(s),...",[]
2,5dbaf08d29dd38ca5e0379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,30/10/2019,11:02:08,4031875,True,Venta,False,...,,,0,0,,False,0.0,"[Balcón, Citófono, Closet, Cocina Integral, Sh...","[Ascensor, Canchas Deportivas, En conjunto cer...","[Área Urbana, Zona Residencial]"
3,5dbaf08d9319faf5bb0379ee,https://www.fincaraiz.com.co/casa-en-venta/med...,31/10/2019,09:22:46,25/10/2019,01:36:45,5027509,True,Venta,False,...,,,0,0,,False,0.0,"[Balcón, Calentador, Closet, Instalación de ga...","[En conjunto cerrado, Garaje / Parqueadero(s),...",[]
4,5dbaf08d9753b6ebb60379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,23/10/2019,16:50:18,5022378,True,Venta,False,...,Bueno,9 a 15 años,0,0,,False,167000.0,"[Balcón, Baño Auxiliar, Baño Independiente, Ba...","[Acceso Pavimentado, Árboles frutales, Asador,...","[Parques cercanos, Supermercados / C.Comercial..."
5,5dbaf08e9711211b070379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,31/10/2019,11:01:37,5046531,True,Venta,False,...,,,0,0,,False,0.0,"[Calentador, Cocina Integral, Estudio, Instala...","[Canchas Deportivas, En conjunto cerrado, Gara...","[Parques cercanos, Sobre vía principal, Superm..."
6,5dbaf08ee9cf0f27870379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,18/10/2019,20:46:06,4553699,True,Venta,False,...,Excelente,,3,0,,False,240000.0,"[Balcón, Calentador, Closet, Sauna / Turco / J...","[En conjunto cerrado, Garaje / Parqueadero(s),...",[]
7,5dbaf08ef5c8a2f4480379ee,https://www.fincaraiz.com.co/casa-en-venta/med...,31/10/2019,09:22:46,29/10/2019,15:38:25,5039307,True,Venta,False,...,Bueno,9 a 15 años,3,0,,False,100000.0,"[Baño Auxiliar, Cocina Integral, Comedor auxil...","[En conjunto cerrado, Jardín, Parqueadero Visi...",[Zona Residencial]
8,5dbaf08e190920582f0379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,28/10/2019,21:24:04,4519287,True,Venta,False,...,,,0,0,,False,0.0,"[Aire Acondicionado, Alarma, Balcón, Baño Auxi...","[Canchas Deportivas, Circuito cerrado de TV, E...","[Colegios / Universidades, Parques cercanos, S..."
9,5dbaf08ea92abb2dc00379ee,https://www.fincaraiz.com.co/apartamento-en-ve...,31/10/2019,09:22:46,23/10/2019,15:09:37,5021843,True,Venta,False,...,Excelente,1 a 8 años,3,0,,False,445000.0,"[Balcón, Citófono, Cocina Integral, Cocina tip...","[Ascensor, Portería / Recepción]","[Colegios / Universidades, Parques cercanos, S..."
